In [128]:
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn

In [129]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(in_features=50, out_features=3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [130]:
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def __repr__(self):
        return f"mean: {self.mean}, std:{self.std}, epsilon:{self.epsilon}"

In [131]:
irisDataset_corto = pd.read_csv("iris_corto.data", names=["sepallength","sepalwidth","petallength","petalwidth"])

In [132]:
irisDataset_corto

,sepallength,sepalwidth,petallength,petalwidth
0,5.0,3.4,1.5,0.2
1,4.4,2.9,1.4,0.2
2,4.9,3.1,1.5,0.1
3,5.4,3.7,1.5,0.2
4,4.8,3.4,1.6,0.2
5,7.0,3.2,4.7,1.4
6,6.4,3.2,4.5,1.5
7,7.6,3.0,6.6,2.1


In [133]:
modelo_cargado = Model(4)
modelo_cargado.load_state_dict(torch.load("pesos.pt"))
modelo_cargado.eval()

torch.set_printoptions(sci_mode=False)

In [134]:
sc = StandardScaler(mean = torch.tensor([5.84, 3.05, 3.76, 1.20]), std = torch.tensor([0.83, 0.43, 1.76, 0.76]), epsilon=1e-7)

In [135]:
s = irisDataset_corto.iloc[:,0:4].values
x_tensor = torch.tensor(s)
x_tensor = x_tensor.to(torch.float)

XScalada = sc.transform(x_tensor)

In [136]:
modelo_cargado(XScalada)

tensor([[    1.000000,     0.000000,     0.000000],
        [    1.000000,     0.000000,     0.000000],
        [    0.999995,     0.000005,     0.000000],
        [    1.000000,     0.000000,     0.000000],
        [    1.000000,     0.000000,     0.000000],
        [    0.000001,     0.999999,     0.000000],
        [    0.000003,     0.999997,     0.000000],
        [    0.000000,     0.000000,     1.000000]],
       grad_fn=<SoftmaxBackward0>)